In [38]:
import json
import copy
import re
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [39]:
# cesta k annotacim, pokud jsou data neaugmentovana druha cesta neni potreba
annotation_path = '../CoralClef2020/annotations/annotation.json'
augmented_annotation_path = '../CoralClef2020/annotations/annotation_augment.json'

In [40]:
with open(annotation_path) as json_file:
    data = json.load(json_file)

In [41]:
images_all = dict()
for i in data['images']:
    images_all.update({i['id']: []})

bb_cls_all = dict.fromkeys(tuple(range(13)), 0) 

for c in data['annotations']:
    bb_cls_all[c['category_id']] += 1
    images_all[c['image_id']].append(c['category_id'])

    


In [42]:
print('images:', len(data['images']))
print('bb:', len(data['annotations']))
print('avg per img:', len(data['annotations'])/len(data['images']))
print()
df = pd.DataFrame({
    'class_id': list(bb_cls_all.keys()),
    'occurrence': list(bb_cls_all.values())
    })
     
print(df)

images: 440
bb: 12082
avg per img: 27.45909090909091

    class_id  occurrence
0          0        1181
1          1         198
2          2        1642
3          3         946
4          4          21
5          5         177
6          6         223
7          7        5663
8          8          90
9          9        1691
10        10         139
11        11          19
12        12          92


In [43]:
# vypocet kolik instanci ma byt ve validacnim setu
val_coef = 0.10
val_img_num = int(len(data['images'])*val_coef)
val_cls_num = [int(round(i * val_coef)) for i in list(bb_cls_all.values())]

print(val_cls_num)
print(val_img_num)

[118, 20, 164, 95, 2, 18, 22, 566, 9, 169, 14, 2, 9]
44


In [44]:
# rozdeleni obrazku
images_val = dict()
images_train = copy.deepcopy(images_all)


while(True):
    min_i = np.argmin(val_cls_num)
    min_n = np.min(val_cls_num)
    if(min_n <= 0):
        val_cls_num[min_i] = float('Inf')
    if(min_n == float('Inf')):
        break

    for img in images_train:
        if min_i in images_train[img]:
            val = images_train.pop(img)
            images_val.update({img: val})
            for k in val:
                val_cls_num[k] -= 1
            break
    
print(len(images_val))
print(val_cls_num)

69
[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]


In [45]:
# pocet instanci v jednotlivich mnozinach
bb_cls_val = dict.fromkeys(tuple(range(13)), 0) 
for img in images_val:
    for cls in images_val[img]:
        bb_cls_val[cls] += 1
        
bb_cls_train = dict.fromkeys(tuple(range(13)), 0) 
for img in images_train:
    for cls in images_train[img]:
        bb_cls_train[cls] += 1

In [46]:
df = pd.DataFrame({
    'class_id': list(bb_cls_all.keys()),
    'occurrence_all': list(bb_cls_all.values()),
    'occurrence_train': list(bb_cls_train.values()),
    'train [%]' : [round(x/y * 100) for x, y in zip(list(bb_cls_train.values()), list(bb_cls_all.values()))],
    'occurrence_val': list(bb_cls_val.values()),
    'val [%]' : [round(x/y * 100) for x, y in zip(list(bb_cls_val.values()), list(bb_cls_all.values()))]
    })
     
print(df.to_string(index=False))

class_id  occurrence_all  occurrence_train  train [%]  occurrence_val  val [%]
       0            1181               893         76             288       24
       1             198               172         87              26       13
       2            1642              1364         83             278       17
       3             946               816         86             130       14
       4              21                18         86               3       14
       5             177               153         86              24       14
       6             223               179         80              44       20
       7            5663              4459         79            1204       21
       8              90                79         88              11       12
       9            1691              1514         90             177       10
      10             139               107         77              32       23
      11              19                16         8

In [47]:
val_id = list(images_val.keys())
val_id.sort()
train_id = list(images_train.keys())
train_id.sort()

annot_val = {'images': [], 'categories': [], 'annotations': []}
annot_train = {'images': [], 'categories': [], 'annotations': []}

In [48]:
for img in data['images']:
    if img['id'] in val_id:
        annot_val['images'].append(img)
    else:
        annot_train['images'].append(img)
        
for an in data['annotations']:
    if an['image_id'] in val_id:
        annot_val['annotations'].append(an)
    else:
        annot_train['annotations'].append(an)

In [49]:
with open('../CoralClef2020/annotations/validation_annot.json', 'w') as file:
    json.dump(annot_val, file)
    
with open('../CoralClef2020/annotations/train_annot.json', 'w') as file:
    json.dump(annot_train, file)

## rozdeleni augmentovanych dat

In [ ]:
with open(augmented_annotation_path) as json_file:
    augment = json.load(json_file)

In [ ]:
annot_augment_train = {'images' : [], 'annotations' : []}
annot_augment_val = {'images' : [], 'annotations' : []}

for a in annot_val['images']:
    name = a['file_name'].split('.')[0]
    for au in augment['images']:
        name2 = au['file_name'].split('.')[0]
        id_img = au['id']
        if re.findall(name, name2):
            annot_augment_val['images'].append(au)
            for an in augment['annotations']:
                if(id_img == an['image_id']):
                    annot_augment_val['annotations'].append(an)
    
for a in annot_train['images']:
    name = a['file_name'].split('.')[0]
    for au in augment['images']:
        name2 = au['file_name'].split('.')[0]
        id_img = au['id']
        if re.findall(name, name2):
            annot_augment_train['images'].append(au)
            for an in augment['annotations']:
                if(id_img == an['image_id']):
                    annot_augment_train['annotations'].append(an)    

annot_val = annot_augment_val
annot_train = annot_augment_train

In [ ]:
types_of_substrates = ['c_hard_coral_branching', 'c_hard_coral_submassive', 'c_hard_coral_boulder',
                       'c_hard_coral_encrusting', 'c_hard_coral_table', 'c_hard_coral_foliose', 'c_hard_coral_mushroom',
                       'c_soft_coral', 'c_soft_coral_gorgonian', 'c_sponge', 'c_sponge_barrel', 'c_fire_coral_millepora',
                       'c_algae_macro_or_leaves']

annot_val.update({'categories': []})
annot_train.update({'categories': []})
k = 0
for c in types_of_substrates:
    annot_val['categories'].append({"supercategory": c,"id": k,"name": c})
    annot_train['categories'].append({"supercategory": c,"id": k,"name": c})
    k = k + 1


for a in annot_val['annotations']:
    a['bbox'][0] = int(a['bbox'][0])
    a['bbox'][1] = int(a['bbox'][1])
    a['bbox'][2] = int(a['bbox'][2])
    a['bbox'][3] = int(a['bbox'][3])
    
for a in annot_train['annotations']:
    a['bbox'][0] = int(a['bbox'][0])
    a['bbox'][1] = int(a['bbox'][1])
    a['bbox'][2] = int(a['bbox'][2])
    a['bbox'][3] = int(a['bbox'][3])

In [51]:
with open('../CoralClef2020/annotations/validation_annot_aug.json', 'w') as file:
    json.dump(annot_val, file)
    
with open('../CoralClef2020/annotations/train_annot_aug.json', 'w') as file:
    json.dump(annot_train, file)